<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/twadr_0_mixed_pubmed_bigtweet_twadr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
Unpacking objects: 100% (9/9), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,13,insomnia
1,57,hypertrophic scarring
2,84,paraplegic
3,93,shooting
4,97,screwing with my short term memory


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-0.test.csv",header=None)
test.head()

,0,1
0,13,insomniac
1,60,cigarette withdrawal
2,108,cramps
3,109,make me drunker
4,133,nightmares


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_twadr.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'was',
 'diclofenac',
 'were',
 'on',
 'is',
 'rt',
 'pain',
 'sodium',
 'by',
 'drug',
 'that',
 'as',
 'at',
 'i',
 'or',
 'patients',
 'this',
 'from',
 'you',
 'disease',
 'be',
 'mg',
 'increased',
 'group',
 'my',
 'study',
 'are',
 'not',
 'mood',
 'treatment',
 'an',
 'it',
 'after',
 'have',
 'me',
 'amp',
 'release',
 'gluten',
 'p',
 'disorder',
 ',',
 'all',
 'effect',
 'abnormal',
 'free',
 'withdrawal',
 'symptoms',
 'we',
 'than',
 'no',
 'decreased',
 'weight',
 'new',
 'out',
 'more',
 'anxiety',
 'your',
 'compared',
 'using',
 'effects',
 'day',
 'time',
 'groups',
 'up',
 'our',
 'about',
 'two',
 'both',
 'inflammatory',
 'between',
 'blood',
 'h',
 'its',
 'drugs',
 'significantly',
 'can',
 'acute',
 'like',
 'skin',
 'these',
 'potassium',
 'disorders',
 'had',
 'dose',
 'anti',
 'but',
 'significant',
 'use',
 'w

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.724776,4.204413,0.343263,22:33


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.000105,3.825232,0.378916,22:38


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.726514,3.780101,0.383834,22:50


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.672726,3.746495,0.387953,22:44


In [25]:
learn.save_encoder('pubmed_cadec_twadrl_mixed_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_twadrl_mixed_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.200335,6.361985,0.121739,00:22


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.232811,4.962025,0.139130,00:25


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.749725,4.835821,0.147826,01:03


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.410700,4.499399,0.191304,01:08
1,5.172761,4.344936,0.208696,01:07


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.900448,4.139490,0.226087,01:05
1,4.724867,4.035728,0.252174,01:05


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.547280,3.900797,0.286957,01:06
1,4.358992,3.812672,0.339130,01:04


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.158999,3.670720,0.295652,01:06
1,4.016132,3.586672,0.330435,01:09


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.824432,3.479483,0.339130,01:07
1,3.678091,3.451567,0.365217,01:03


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.463684,3.408813,0.339130,01:05
1,3.397720,3.461498,0.321739,01:07
2,3.210050,3.425339,0.330435,01:09
3,3.063501,3.415547,0.339130,01:02


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.904715,3.346993,0.347826,01:06
1,2.823420,3.304632,0.347826,01:06
2,2.668540,3.365725,0.339130,01:05
3,2.531029,3.376406,0.347826,01:02


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.398787,3.374352,0.339130,01:03
1,2.328070,3.333186,0.365217,01:05
2,2.163985,3.363324,0.365217,01:03
3,2.009849,3.322783,0.391304,01:08


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.871330,3.336455,0.347826,01:11
1,1.858968,3.435100,0.347826,01:04
2,1.716675,3.308690,0.373913,01:08
3,1.581222,3.376554,0.373913,01:06


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.475009,3.394918,0.400000,01:06
1,1.455737,3.483049,0.373913,01:08
2,1.331118,3.496422,0.365217,01:05
3,1.223932,3.470602,0.347826,01:07


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.078375,3.496231,0.391304,01:06
1,1.135011,3.557838,0.382609,01:06
2,1.039755,3.593024,0.373913,01:06
3,0.977833,3.552427,0.373913,01:07


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.890423,3.587836,0.356522,01:11
1,0.909924,3.665102,0.408696,01:09
2,0.827548,3.650471,0.382609,01:11
3,0.768857,3.681746,0.382609,01:10


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.718711,3.753326,0.382609,01:08
1,0.689414,3.844860,0.373913,01:08


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.630297,3.921396,0.356522,01:07
1,0.639975,4.052668,0.382609,01:07


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.628173,4.052601,0.347826,01:03


In [44]:
# save the best model

learn.save_encoder('pubmed_cadec_twadrl_mixed_fold0')

In [48]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.587421,4.078191,0.356522,01:06


# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[393, 1597, 265, 1578, 1184, 936, 483, 189, 231, 231, 233, 233, 233, 233, 249, 249, 1724, 393, 2168, 1319, 1529, 1099, 644, 1711, 749, 749, 452, 792, 792, 792, 813, 852, 852, 749, 1356, 1099, 974, 978, 978, 978, 978, 978, 1884, 418, 1007, 2113, 1669, 1767, 1089, 1099, 852, 1925, 1136, 1136, 908, 1204, 1295, 1099, 1942, 441, 1265, 1930, 1295, 195, 518, 1308, 1340, 1265, 1340, 1265, 1340, 1340, 1822, 1484, 2077, 1526, 1711, 1925, 1597, 1597, 1597, 1597, 1597, 1599, 767, 1942, 1610, 418, 1669, 852, 1682, 1915, 1711, 1099, 1767, 1767, 1340, 1768, 1073, 233, 1718, 2191, 1767, 1718, 1787, 1787, 2032, 1787, 1787, 1787, 1099, 1718, 1815, 1815, 1767, 1940, 1877, 1884, 825, 1787, 1930, 1936, 233, 1940, 1942, 1942, 2002, 2109, 899, 2018, 1787, 452, 1822, 1877, 974, 1787, 2032, 2147, 974, 441, 1787, 824, 824]
[13, 60, 108, 109, 133, 154, 187, 189, 231, 231, 233, 233, 233, 233, 249, 249, 265, 393, 418, 520, 561, 561, 644, 749, 749, 749, 772, 792, 792, 792, 813, 852, 852, 887, 908, 974, 974,

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

67
0.46853146853146854
